In [1]:
import tensorflow

print(tensorflow.__version__)

2.6.0


In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
#샘플 데이터 불러옴
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
91274,Tom thought Mary was alone.,Tom pensait que Marie était seule.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
155185,I was going to ask you the same thing.,J'allais vous demander la même chose.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
97745,Tell me about your children.,Parle-moi de tes enfants.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
103938,I'm just as tired as you are.,Je suis aussi fatigué que toi.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
89535,Of course Tom will be late.,Evidemment Tom sera en retard.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...


In [4]:
# 3번째 열은 불필요하므로 제거하고, 훈련 데이터는 33000개의 샘플로 줄임
lines = lines[['eng', 'fra']][:33000] # 33000개 샘플 사용 3000개는 추후 테스트 샘플로 사용
lines.sample(5)

,eng,fra
20962,Tom can't see us.,Tom ne peut pas nous voir.
4440,Are you deaf?,Êtes-vous sourde ?
12942,You're kidding!,Sans blague !
32528,Tom drank too much.,Tom a trop bu.
27063,We can solve this.,Nous pouvons résoudre ceci.


In [5]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
16531,Turn the fan on.,\t Allume le ventilateur. \n
658,I'm fast.,\t Je suis rapide. \n
25910,That makes me cry.,\t Ça me fait pleurer. \n
16904,What's its name?,\t Quel est son nom ? \n
19845,It's very simple.,\t C'est très simple. \n


In [6]:
# 각 데이터 프레임 칼럼을 리스트화
eng_list = lines['eng'].to_list()
fra_list = lines['fra'].to_list()

In [7]:
eng_list[:5]

['Go.', 'Go.', 'Go.', 'Go.', 'Hi.']

In [8]:
# 1. 구두점을 단어와 분리
import re, string

def punc(s): return re.sub(f"([{string.punctuation}])", r' \1 ', s)

In [9]:
p_el = [punc(i) for i in eng_list]

In [10]:
p_el[:5]

['Go . ', 'Go . ', 'Go . ', 'Go . ', 'Hi . ']

In [11]:
p_fl = [punc(i) for i in fra_list]

In [12]:
p_fl[:5]

['\t Va  !  \n',
 '\t Marche .  \n',
 '\t En route  !  \n',
 '\t Bouge  !  \n',
 '\t Salut  !  \n']

In [13]:
# 소문자로 바꾸고 띄어쓰기 단위로 토큰화 
eng_tokenizer = Tokenizer(char_level=False, filters='', lower =True)   # 단어 단위로 Tokenizer를 생성합니다. 소문자로 바꾸고 필터(구두점제거)
eng_tokenizer.fit_on_texts(p_el)               # 33000개의 행을 가진 p_el의 각 요소들에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(p_el)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[29, 1], [29, 1], [29, 1]]

In [14]:
fra_tokenizer = Tokenizer(char_level=False, filters='', lower =True)   # 단어 단위로 Tokenizer를 생성합니다. 소문자로 바꾸고 필터(구두점제거)
fra_tokenizer.fit_on_texts(p_fl)               # 33000개의 행을 가진 p_fl의 각 요소들에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(p_fl)    # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 76, 10, 2], [1, 345, 3, 2], [1, 27, 486, 10, 2]]

In [15]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4713
프랑스어 단어장의 크기 : 9467


In [16]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 20


In [17]:
#전체적인 통계정보를 한꺼번에 출력

print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4713
프랑스어 단어장의 크기 : 9467
영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 20


In [18]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [19]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 76, 10], [1, 345, 3], [1, 27, 486, 10]]
[[76, 10, 2], [345, 3, 2], [27, 486, 10, 2]]


In [20]:
#데이터의 크기 확인
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10)
프랑스어 입력데이터의 크기(shape) : (33000, 20)
프랑스어 출력데이터의 크기(shape) : (33000, 20)


In [21]:
print(encoder_input[0])

[29  1  0  0  0  0  0  0  0  0]


In [22]:
# # 벡터화 원핫 인코딩
# encoder_input = to_categorical(encoder_input)
# decoder_input = to_categorical(decoder_input)
# decoder_target = to_categorical(decoder_target)
# print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
# print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
# print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

In [23]:
# validation 생성 3000건 나머지 학습데이터로
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 10)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 20)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 20)


In [24]:
#프랑수아 숄레의 케라스의 seq2seq 구현 가이드 게시물인 A ten-minute introduction to sequence-to-sequence learning를 참고
#도구 임포트

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model


In [47]:
#임베딩 층 구현
from tensorflow.keras.layers import Input, Embedding, Masking

# 인코더에서 사용할 임베딩 층 사용 예시(영어) 임베딩 벡터의 차원(word_vec_dim)을 8으로 설정 하이퍼 파라미터
word_vec_dim = 512
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(eng_vocab_size, word_vec_dim )(encoder_inputs)
# hidden size가 16인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state=True)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [48]:
#디코더 설계
# 입력 텐서 생성. word_vec_dim 16 동일

decoder_inputs = Input(shape=(None,))
dec_emb =  Embedding(fra_vocab_size, word_vec_dim )(decoder_inputs)
# hidden size가 16인 디코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(dec_emb, initial_state = encoder_states)

In [49]:
#디코더의 출력층 설계
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [50]:
#매 time step마다의 다중 클래스 분류 문제이므로 프랑스어 단어장으로부터 한 가지 문자만 선택하도록 합니다. Dense의 인자로 프랑스어 단어장의 크기를 기재하고, 활성화 함수로 소프트맥스 함수를 사용. 최종적으로 인코더와 디코더를 연결해서 하나의 모델로 만들어줍니다. Model의 Input과 Output의 정의를 유심히 살펴 주세요.

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 512)    2413056     input_9[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 512)    4847104     input_10[0][0]                   
____________________________________________________________________________________________

In [51]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor= 'val_loss', patience= 3)

history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
                    validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
                    batch_size=256, epochs=100, callbacks = earlystopping)

Epoch 1/100
118/118 [==============================] - 9s 53ms/step - loss: 1.8971 - val_loss: 1.5633
Epoch 2/100
118/118 [==============================] - 6s 48ms/step - loss: 1.1930 - val_loss: 1.3590
Epoch 3/100
118/118 [==============================] - 6s 49ms/step - loss: 1.0162 - val_loss: 1.2391
Epoch 4/100
118/118 [==============================] - 6s 49ms/step - loss: 0.8990 - val_loss: 1.1385
Epoch 5/100
118/118 [==============================] - 6s 49ms/step - loss: 0.8162 - val_loss: 1.0697
Epoch 6/100
118/118 [==============================] - 6s 50ms/step - loss: 0.7517 - val_loss: 1.0291
Epoch 7/100
118/118 [==============================] - 6s 50ms/step - loss: 0.6972 - val_loss: 0.9925
Epoch 8/100
118/118 [==============================] - 6s 50ms/step - loss: 0.6501 - val_loss: 0.9570
Epoch 9/100
118/118 [==============================] - 6s 50ms/step - loss: 0.6083 - val_loss: 0.9441
Epoch 10/100
118/118 [==============================] - 6s 51ms/step - loss: 0.571

In [52]:
#테스트 단계의 디코더의 동작 순서
#인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다.
# 토큰인 '\t'를 디코더에 입력한다.
# 이전 time step의 출력층의 예측 결과를 현재 time step의 입력으로 한다.
# 3을 반복하다가 토큰인 '\n'가 예측되면 이를 중단한다.

#모델 테스트하기
#인코더 정의
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 512)         2413056   
_________________________________________________________________
lstm_4 (LSTM)                [(None, 256), (None, 256) 787456    
Total params: 3,200,512
Trainable params: 3,200,512
Non-trainable params: 0
_________________________________________________________________


In [53]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(dec_emb, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [54]:
#출력층 재설계
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 512)    4847104     input_10[0][0]                   
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 256)]        0                                            
____________________________________________________________________________________________

In [55]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [56]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # 에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1)) 
    target_seq[0, 0] = fra2idx['\t']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # 에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장     
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [57]:
for seq_index in [3,50,100,300,1001, 134, 345, 332, 234, 141]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장:  Bouge ! 
번역기가 번역한 문장:  va ! 
-----------------------------------
입력 문장: Hello!
정답 문장:  Bonjour ! 
번역기가 번역한 문장:  bonjour ! 
-----------------------------------
입력 문장: Got it?
정답 문장:  T'as capté ? 
번역기가 번역한 문장:  est - ce que ça a ét
-----------------------------------
입력 문장: Hang on.
정답 문장:  Tiens bon ! 
번역기가 번역한 문장:  attendez . 
-----------------------------------
입력 문장: Here's $5.
정답 문장:  Voilà cinq dollars. 
번역기가 번역한 문장:  voici là . 
-----------------------------------
입력 문장: No way!
정답 문장:  Hors de question ! 
번역기가 번역한 문장:  sans question ! 
-----------------------------------
입력 문장: I'm hit!
정답 문장:  Je suis touché ! 
번역기가 번역한 문장:  je suis touché ! 
-----------------------------------
입력 문장: I stood.
정답 문장:  Je me suis tenu debout. 
번역기가 번역한 문장:  je me suis mis à pleure
-----------------------------------
입력 문장: Get out!
정답 문장:  Dégage ! 
번역기가 번역한 문장:  sortez ! 
-----------------------------------
입력 문장: Thanks.
정답 문장:  Je te reme

-----------------------------------
1. 입력 문장: Go.
정답 문장:  Bouge ! 
번역기가 번역한 문장:  va !  
##### 정답 문장과 다름 프랑스어 의미도 다름, 그러나 스페인어로 간다는 의미임 va ! 학습시에 스페인어가 더 많이 쓰였나?
-----------------------------------
2. 입력 문장: Hello!
정답 문장:  Bonjour ! 
번역기가 번역한 문장:  bonjour !  
##### 정답! 
-----------------------------------
3. 입력 문장: Got it?
정답 문장:  T'as capté ? 
번역기가 번역한 문장:  est - ce que ça a ét  
#####  의미 같음
-----------------------------------
4. 입력 문장: Hang on.
정답 문장:  Tiens bon ! 
번역기가 번역한 문장:  attendez .   
#####  의미 같음
-----------------------------------
5. 입력 문장: Here's $5.
정답 문장:  Voilà cinq dollars. 
번역기가 번역한 문장:  voici là . 
##### 정답 문장과 다름, 의미는 비슷하나 5달러가 빠져있음
-----------------------------------
6. 입력 문장: No way!
정답 문장:  Hors de question ! 
번역기가 번역한 문장:  sans question !  
##### 의미 같음
-----------------------------------
7. 입력 문장: I'm hit!
정답 문장:  Je suis touché ! 
번역기가 번역한 문장:  je suis touché ! 
##### 정답!
-----------------------------------
8. 입력 문장: I stood.
정답 문장:  Je me suis tenu debout. 
번역기가 번역한 문장:  je me suis mis à pleure  
##### 정답 문장과 다름, 의미도 달라보임
----------------------------------- 
9. 입력 문장: Get out!
정답 문장:  Dégage ! 
번역기가 번역한 문장:  sortez !       
##### 의미 같음
-----------------------------------
10. 입력 문장: Thanks.
정답 문장:  Je te remercie. 
번역기가 번역한 문장:  merci !  
##### 의미 같음

- #### 영어에서 바로 프랑스어 1:1 대응이 아닌 
- #### 영어 -> X -> 프랑스어로 인코딩, 디코딩 그리고 출력을 설계함
- #### 모델이 학습 후 마치 영어의 의미를 이해하여 프랑스어로 바꿔주는 듯한 느낌을 받았다

# 회고

- 지난 ex 6에서는 직접 구두점들을 설정하고 다루는 코드를 작성하여 사용하였으나,
- 이번에는 string 내장 함수를 사용하여 구두점 작업을 하였다.
- sentence to sentence 의 번역기 코드를 작성해 볼 수 있었다.
- 직접 임베딩 레이어를 추가하여 단어의 임베딩 깊이를 조절해 볼 수 있었다.
- 영어를 인코딩 하여 어떤 x로 바꾸고 그 x를 디코딩하여 프랑스어로 바꾸는 알고리즘으로 러닝 모델을 작성해 보았다. 
- Total params를 최대한으로 줄여서 모델 학습을 시켜보려고 하였으나 클라우드 서버가 계속 다운이 되어 진행시키지 못했다.
- 코드 자체의 문제인지 많은사람이 한꺼번에 코드를 돌려서 서버 리소스의 문제인지 확인을 하지 못하였다.
-  코드 자체에서 임베딩을 할때 원핫 인코딩으로 30000개의 단어로 30000X30000의 인코딩이 생겨 발생한 것으로 확인 
-  원핫 인코딩을 하지않고, encoding layer 설계할때에 인스턴스로 인코딩을 하는 레이어를 추가하여 학습시킴 
- 지난번 데이터톤 때에도 느꼈지만 장비(GPU TPU등 연산장치 또는 머신러닝을 구현하는데 필요한 부수 장비)의 성능 및 발전도 머신러닝을 연구하고 발전시키는데 아주 중요한 부분임을 새삼 깨달았다
- colab또는 좋은 머신러닝구현 환경을 사용하도록 준비를 해야겠다.